In [ ]:
!pip install node2vec
!pip install --upgrade gensim
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
import numpy as np 
import random
random.seed(2022)
np.random.seed(2022)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_10000_9.csv to data_10000_9 (1).csv
Saving edge_index_data_9.csv to edge_index_data_9 (1).csv


In [ ]:
uploaded2 = files.upload()

Saving deepwalk_10d_x9.model to deepwalk_10d_x9 (1).model
Saving Node2Vec_10d_x9.model to Node2Vec_10d_x9 (1).model


In [ ]:
import io
df=pd.read_csv(io.BytesIO(uploaded['data_10000_9.csv']),index_col=0)

from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size = 0.2,random_state=42)
df

,X0,X1,X2,X3,X4,X5,X6,X7,X8,T,y,tau,b,e
0,0.609399,0.617459,0.226921,0.004359,0.661646,0.587542,0.426148,0.705872,0.073834,1,1.006577,0.613429,1.409735,0.900000
1,0.835325,0.171336,0.723448,0.643941,0.351913,0.923063,0.628349,0.513777,0.865254,0,2.414745,0.503331,1.354386,0.434631
2,0.640500,0.000786,0.625402,0.270436,0.083907,0.505043,0.057420,0.315679,0.406920,0,2.194673,0.320643,0.345424,0.100000
3,0.108567,0.594597,0.530164,0.261543,0.733564,0.454656,0.623660,0.301259,0.325126,0,0.845496,0.351582,0.831559,0.201414
4,0.534229,0.438480,0.679938,0.074687,0.071026,0.760965,0.131953,0.041444,0.336757,1,2.220674,0.486354,0.846220,0.671265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.886510,0.195066,0.283373,0.465533,0.482518,0.099990,0.010150,0.900044,0.546843,1,1.272802,0.540788,1.317584,0.516938
9996,0.993493,0.414716,0.167049,0.898168,0.883686,0.555195,0.939795,0.446944,0.670944,1,2.604282,0.704104,2.523766,0.900000
9997,0.864848,0.326439,0.325487,0.126372,0.429563,0.997772,0.174929,0.967537,0.588439,1,1.822956,0.595644,1.177203,0.775139
9998,0.843089,0.690255,0.432493,0.709480,0.033376,0.687961,0.781540,0.317079,0.218253,1,2.139239,0.766672,1.702327,0.900000


In [ ]:
col=df.columns
y_train=torch.from_numpy(np.array((df_train['y']))).reshape(df_train.shape[0],1).to(torch.float32)
y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[col[:10]])).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[col[:10]])).to(torch.float32)

train_reverse=df_train[col[:10]].copy()
train_reverse["T"]=1-train_reverse["T"]
train_reverse=torch.from_numpy(np.array(train_reverse)).to(torch.float32)
test_reverse=df_test[col[:10]].copy()
test_reverse["T"]=1-test_reverse["T"]
test_reverse=torch.from_numpy(np.array(test_reverse)).to(torch.float32)

In [ ]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl 

In [ ]:
edge_index=pd.read_csv(io.BytesIO(uploaded['edge_index_data_9.csv']))
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))

In [ ]:
from gensim.models import word2vec
model_dw=word2vec.Word2Vec.load("deepwalk_10d_x9.model")
model_n2v=word2vec.Word2Vec.load("Node2Vec_10d_x9.model")

lst_dw=[]
lst_n2v=[]
for i in range(10):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_n2v)

10

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, download_url

# create Mytrain Data
class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            t=torch.zeros(10,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(10): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 
              
            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytest Data
class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
   
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]
            t=torch.zeros(10,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(10): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# create Mytrain_reverse Data
class Mytrain_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        
    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]
    
    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(train_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =train_reverse[i]
            t=torch.zeros(10,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(10): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 

            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)
            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


# create Mytest_reverse Data
class Mytest_reverse(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y
        

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(test_reverse.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =test_reverse[i]
            t=torch.zeros(10,11) # 11 dimensions = 1+10 = node number, node embedding
            
            for j in range(10): # for j-th feature
              t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # current node (1d)
                                                     list(lst_dw[int(j)]), # node embedding (10d), prior embeddings from DeepWalk/Node2Vec     
                                                    ))) 

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
Mydata_train=Mytrain(".\mydata\MYdata_train")
Mydata_test=Mytest(".\mydata\MYdata_test")
Mytrain_reverse=Mytrain_reverse(".\mydata\MYtrain_reverse")
Mytest_reverse=Mytest_reverse(".\mydata\MYtest_reverse")

# GCN

In [ ]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(11, 64)
        self.conv2 = GCNConv(64, 11)
        self.f1 = torch.nn.Linear(110,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,110)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [ ]:

def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    

te_train = {}
te_valid = {}
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GCN ATE % Error",i]=np.mean(train_error)
    te_train["GCN MSE",i]=np.mean(train_mse)
    te_train["GCN KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GCN ATE % Error",i]=np.mean(valid_error)
    te_valid["GCN MSE",i]=np.mean(valid_mse)
    te_valid["GCN KL Divergence",i]=np.mean(valid_kl)

R2:-0.575677  MSE:2.227177  KL:0.365370  MAE:1.196908  RMSE:1.492373  CVRMSE:0.991156  train_loss:
2.2271769256591796
R2:-0.077784  MSE:1.564907  KL:1.809186  MAE:1.007018  RMSE:1.250962  CVRMSE:0.881215  test_loss:
0.0031265507340431215
R2:0.002979  MSE:1.409262  KL:0.912814  MAE:0.950060  RMSE:1.187123  CVRMSE:0.788426  R2:0.044147  MSE:1.351073  KL:0.970307  MAE:0.928079  RMSE:1.162357  CVRMSE:0.771977  R2:0.081768  MSE:1.297897  KL:0.894165  MAE:0.907437  RMSE:1.139253  CVRMSE:0.756632  R2:0.116988  MSE:1.248113  KL:0.608174  MAE:0.890991  RMSE:1.117190  CVRMSE:0.741979  R2:0.135449  MSE:1.222020  KL:0.429313  MAE:0.879694  RMSE:1.105450  CVRMSE:0.734183  R2:0.144016  MSE:1.209911  KL:0.391592  MAE:0.876326  RMSE:1.099959  CVRMSE:0.730536  R2:0.150635  MSE:1.200554  KL:0.395817  MAE:0.872714  RMSE:1.095698  CVRMSE:0.727706  R2:0.151080  MSE:1.199926  KL:0.378153  MAE:0.872130  RMSE:1.095411  CVRMSE:0.727515  R2:0.154128  MSE:1.195618  KL:0.369714  MAE:0.871071  RMSE:1.093443  CVRMS

In [ ]:
te_train,te_valid

({('GCN ATE % Error', 0): 0.2420790983477119,
  ('GCN ATE % Error', 1): 0.07878974893677748,
  ('GCN ATE % Error', 2): 0.1707887024721535,
  ('GCN ATE % Error', 3): 0.13420300072255298,
  ('GCN ATE % Error', 4): 0.10802688991398735,
  ('GCN KL Divergence', 0): 0.16428169316554292,
  ('GCN KL Divergence', 1): 0.04050678553295198,
  ('GCN KL Divergence', 2): 0.0992463019782772,
  ('GCN KL Divergence', 3): 0.06686006472917819,
  ('GCN KL Divergence', 4): 0.05381154226722651,
  ('GCN MSE', 0): 0.07944656930462664,
  ('GCN MSE', 1): 0.06599758130685263,
  ('GCN MSE', 2): 0.06751150642989465,
  ('GCN MSE', 3): 0.0686010019424961,
  ('GCN MSE', 4): 0.06840242532233463},
 {('GCN ATE % Error', 0): 0.37444532543038167,
  ('GCN ATE % Error', 1): 0.20979524098329927,
  ('GCN ATE % Error', 2): 0.3032235923219577,
  ('GCN ATE % Error', 3): 0.2617007334171957,
  ('GCN ATE % Error', 4): 0.24207526605884544,
  ('GCN KL Divergence', 0): 0.3316322882815698,
  ('GCN KL Divergence', 1): 0.1796567932139104,

# GAT

In [ ]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.gat1=GATConv(in_channels=11,out_channels=8,heads=8,dropout=0.6)
        self.gat2=GATConv(in_channels=64,out_channels=11,heads=1,dropout=0.6)
        self.f1 = torch.nn.Linear(110,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,110)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context

def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in train_loader:
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()
    
    
    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()

    
    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

    
for i in range(5):
    train_mse=[]
    train_kl=[]
    train_error=[]

    valid_mse=[]
    valid_kl=[]
    valid_error=[]

    num_epochs = 2560
    batch_size = 512
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
    crit = F.mse_loss
    train_loader = DataLoader(Mydata_train, batch_size=batch_size)
    test_loader = DataLoader(Mydata_test, batch_size=512)
    reverse_train = DataLoader(Mytrain_reverse, batch_size=512)
    reverse_test = DataLoader(Mytest_reverse, batch_size=512)
    for epoch in range(num_epochs):
        loss,r2,mse,kl=train()
        if epoch %64==0:      
            print('train_loss:')
            print(loss)

            loss,r2,mse,kl=val()
            print('test_loss:')
            print(loss)
  
    y_predicted = []
    for data in train_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_train["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_train:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_train["y_reversed"]=y_reversed
    t0=df_train[df_train["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_train[df_train["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    train_mse.append(mse)
    train_kl.append(kl)
    train_error.append(ate_error)

    print("Model %d has MSE on trainset:%f" % (i,np.mean(train_mse)),end='  ')
    print("Model %d has KL on trainset:%f" % (i,np.mean(train_kl)),end='  ')
    print("Model %d has ATE error on trainset:%f" % (i,np.mean(train_error)),end='  ')

    te_train["GAT ATE % Error",i]=np.mean(train_error)
    te_train["GAT MSE",i]=np.mean(train_mse)
    te_train["GAT KL Divergence",i]=np.mean(train_kl)

    y_predicted = []
    for data in test_loader:
        loss = 0
        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        y_predicted +=(output).cpu().detach().ravel().tolist()
    df_test["y_hat"]=y_predicted

    y_reversed=[]
    for data in reverse_test:
        data = data.to(device)
        output = model(data)
        y_reversed +=(output).cpu().detach().ravel().tolist()

    df_test["y_reversed"]=y_reversed
    t0=df_test[df_test["T"]==0]
    t0=t0.copy()
    t0.loc[:,"tau_hat"]=t0.loc[:,"y_reversed"]-t0.loc[:,"y_hat"]

    t1=df_test[df_test["T"]==1]
    t1=t1.copy()
    t1.loc[:,"tau_hat"]=t1.loc[:,"y_hat"]-t1.loc[:,"y_reversed"]

    ite_pred=np.concatenate([t1["tau_hat"],t0["tau_hat"]],axis=0)
    ite_true=np.concatenate([t1["tau"],t0["tau"]],axis=0)

    mse=MSE(ite_pred,ite_true)
    kl=kl_divergence(ite_pred,ite_true)
    ate_error=abs((np.mean(ite_pred)/np.mean(ite_true))-1)
    valid_mse.append(mse)
    valid_kl.append(kl)
    valid_error.append(ate_error)

    print("Model %d has MSE on validset:%f" % (i,np.mean(valid_mse)),end='  ')
    print("Model %d has KL on validset:%f" % (i,np.mean(valid_kl)),end='  ')
    print("Model %d has ATE error on validset:%f" % (i,np.mean(valid_error)),end='  ')

    te_valid["GAT ATE % Error",i]=np.mean(valid_error)
    te_valid["GAT MSE",i]=np.mean(valid_mse)
    te_valid["GAT KL Divergence",i]=np.mean(valid_kl)

R2:-0.403907  MSE:1.984384  KL:0.192076  MAE:1.127444  RMSE:1.408682  CVRMSE:0.935573  train_loss:
1.9843843460083008
R2:0.015174  MSE:1.429935  KL:1.807578  MAE:0.951658  RMSE:1.195799  CVRMSE:0.842356  test_loss:
0.00285552316904068
R2:-0.023882  MSE:1.447229  KL:0.592231  MAE:0.960284  RMSE:1.203008  CVRMSE:0.798976  R2:0.039995  MSE:1.356941  KL:0.660605  MAE:0.930826  RMSE:1.164878  CVRMSE:0.773652  R2:0.090899  MSE:1.284990  KL:0.495761  MAE:0.903121  RMSE:1.133574  CVRMSE:0.752861  R2:0.105431  MSE:1.264449  KL:0.342707  MAE:0.896369  RMSE:1.124477  CVRMSE:0.746819  R2:0.110714  MSE:1.256982  KL:0.307314  MAE:0.894125  RMSE:1.121152  CVRMSE:0.744611  R2:0.119079  MSE:1.245159  KL:0.338528  MAE:0.888173  RMSE:1.115867  CVRMSE:0.741101  R2:0.125494  MSE:1.236090  KL:0.336194  MAE:0.885340  RMSE:1.111796  CVRMSE:0.738397  R2:0.133407  MSE:1.224906  KL:0.338265  MAE:0.881589  RMSE:1.106755  CVRMSE:0.735049  R2:0.137615  MSE:1.218958  KL:0.336420  MAE:0.877893  RMSE:1.104064  CVRMSE:

In [ ]:
te_train,te_valid

({('GAT ATE % Error', 0): 0.1554965040800418,
  ('GAT ATE % Error', 1): 0.20156665523194217,
  ('GAT ATE % Error', 2): 0.1460915771441078,
  ('GAT ATE % Error', 3): 0.16787527981805028,
  ('GAT ATE % Error', 4): 0.1602286272803728,
  ('GAT KL Divergence', 0): 0.13144229285471504,
  ('GAT KL Divergence', 1): 0.17235372770944457,
  ('GAT KL Divergence', 2): 0.12428911287200595,
  ('GAT KL Divergence', 3): 0.1884606275769505,
  ('GAT KL Divergence', 4): 0.138406365295056,
  ('GAT MSE', 0): 0.08960401431011189,
  ('GAT MSE', 1): 0.09535263050693596,
  ('GAT MSE', 2): 0.09106808985536737,
  ('GAT MSE', 3): 0.10621296925612035,
  ('GAT MSE', 4): 0.09005327903773568,
  ('GCN ATE % Error', 0): 0.2420790983477119,
  ('GCN ATE % Error', 1): 0.07878974893677748,
  ('GCN ATE % Error', 2): 0.1707887024721535,
  ('GCN ATE % Error', 3): 0.13420300072255298,
  ('GCN ATE % Error', 4): 0.10802688991398735,
  ('GCN KL Divergence', 0): 0.16428169316554292,
  ('GCN KL Divergence', 1): 0.04050678553295198,


In [ ]:
gcn_error=[te_train[i] for i in te_train if 'GCN ATE % Error' in i[0]][1]
gcn_mse=[te_train[i] for i in te_train if 'GCN MSE' in i[0]][1]
gcn_kl=[te_train[i] for i in te_train if 'GCN KL Divergence' in i[0]][1]
gat_error=[te_train[i] for i in te_train if 'GAT ATE % Error' in i[0]][2]
gat_mse=[te_train[i] for i in te_train if 'GAT MSE' in i[0]][2]
gat_kl=[te_train[i] for i in te_train if 'GAT KL Divergence' in i[0]][2]
train_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
train_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
train_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

gcn_error=[te_valid[i] for i in te_valid if 'GCN ATE % Error' in i[0]][1]
gcn_mse=[te_valid[i] for i in te_valid if 'GCN MSE' in i[0]][1]
gcn_kl=[te_valid[i] for i in te_valid if 'GCN KL Divergence' in i[0]][1]
gat_error=[te_valid[i] for i in te_valid if 'GAT ATE % Error' in i[0]][2]
gat_mse=[te_valid[i] for i in te_valid if 'GAT MSE' in i[0]][2]
gat_kl=[te_valid[i] for i in te_valid if 'GAT KL Divergence' in i[0]][2]
valid_summary=pd.DataFrame(columns=[["Abs % Error of ATE","MSE","KL Divergence"]])
valid_summary.loc["S learner(GCN)",:]=[gcn_error,gcn_mse,gcn_kl]
valid_summary.loc["S learner(GAT)",:]=[gat_error,gat_mse,gat_kl]

print("Validation Summary of Embedding(10d) & Unweighted")
valid_summary

Validation Summary of Embedding(10d) & Unweighted


,Abs % Error of ATE,MSE,KL Divergence
S learner(GCN),0.209795,0.072603,0.179657
S learner(GAT),0.208663,0.094822,0.172443


In [ ]:
train_summary.to_csv("Train_embedding(10d)_unweighted_x9.csv")
valid_summary.to_csv("Valid_embedding(10d)_unweighted_x9.csv")